## Stripe: Data Science Intern - Written Project
#### Frank Yue Ying | 2022-03-07

### Raw Data
merchant transaction activity, for merchants that start over a 2 year period (2033-2034). The data spans from 1/1/33 through 12/31/34. Each observation is a transaction amount in cents.
#### Columns
1. ID: [1,1513719] natural order of rows in raw data
2. merchant: unique merchant ID with length of 10 digits & letters
3. time: format as YYYY-MM-DD HH:mm:ss, assuming in the same timezone
4. amount_usd_in_cents: integer number in cents

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load the data
raw_data = pd.read_csv("takehome_ds_written.csv",header = 0,index_col =0)
raw_data.head()

,merchant,time,amount_usd_in_cents
1,faa029c6b0,2034-06-17 23:34:14,6349
2,ed7a7d91aa,2034-12-27 00:40:38,3854
3,5608f200cf,2034-04-30 01:29:42,789
4,15b1a0d61e,2034-09-16 01:06:23,4452
5,4770051790,2034-07-22 16:21:42,20203


In [3]:
#Transform the data: break time into year, month, day
data = raw_data.copy()
def convert_time(row,x):
    if x == 'year':
        return row.split(" ")[0].split("-")[0]
    elif x == 'month':
        return row.split(" ")[0].split("-")[1]
    elif x == 'day':
        return row.split(" ")[0].split("-")[2]

data['year'] = np.vectorize(convert_time)(data['time'], 'year')
data['month'] = np.vectorize(convert_time)(data['time'], 'month')
data['day'] = np.vectorize(convert_time)(data['time'], 'day')

In [44]:
data.dtypes

merchant               object
time                   object
amount_usd_in_cents     int64
year                   object
month                  object
day                    object
dtype: object

In [23]:
#Group by year & month
data_ByYear = data.groupby(["year","month"])["merchant"].nunique()
data_ByYear

year  month
2033  01        390
      02        656
      03        964
      04       1164
      05       1402
      06       1639
      07       1864
      08       2149
      09       2324
      10       2562
      11       2785
      12       3107
2034  01       3227
      02       3490
      03       3931
      04       4079
      05       4437
      06       4630
      07       4789
      08       5175
      09       5273
      10       5740
      11       5955
      12       6126
Name: merchant, dtype: int64

In [11]:
# set up unique merchant-based tracking table 
merchants = data['merchant'].unique().tolist()

In [12]:
len(merchants)
merchant_data = pd.DataFrame(columns = ["id"])
merchant_data.set_index("id")
merchant_data_columns_dict = {}
for year in ['count-2033','count-2034','volume-2033','volume-2034']:
    for month in range(1,13):
        merchant_data[year+"-"+str(month)] = 0
        merchant_data_columns_dict[year+"-"+str(month)] = 0

def query_merchant(dt,merchant,merchant_data_columns_dict):
    merchant_data_values = merchant_data_columns_dict.copy()
    merchant_data = dt.loc[dt['merchant'] == merchant]
    merchant_data = merchant_data.groupby(["year","month"]).agg({"amount_usd_in_cents": [np.count_nonzero, np.sum]}).reset_index()
    for index, row in merchant_data.iterrows():
        merchant_data_values['count-'+str(row['year'].values[0])+"-"+str(int(row['month'].values[0]))] = int(row[2])
        merchant_data_values['volume-'+str(row['year'].values[0])+"-"+str(int(row['month'].values[0]))] = int(row[3])
    return pd.DataFrame([merchant_data_values], columns=merchant_data_values.keys())

for merchant in merchants:
    merchant_dt = query_merchant(data,merchant,merchant_data_columns_dict)
    merchant_dt['id'] = merchant
    merchant_data = pd.concat([merchant_data, merchant_dt], axis=0)

# merchant_data.to_csv("merchant_data.csv", index = False)

In [5]:
# merchant_data = pd.read_csv("merchant_data.csv",header = 0,index_col =0)
merchant_data.head()

,count-2033-1,count-2033-2,count-2033-3,count-2033-4,count-2033-5,count-2033-6,count-2033-7,count-2033-8,count-2033-9,count-2033-10,...,volume-2034-3,volume-2034-4,volume-2034-5,volume-2034-6,volume-2034-7,volume-2034-8,volume-2034-9,volume-2034-10,volume-2034-11,volume-2034-12
id,,,,,,,,,,,,,,,,,,,,,
faa029c6b0,0,0,0,0,0,0,0,11,38,20,...,539391,618490,401317,422276,639421,488288,707890,557971,1071722,585480
ed7a7d91aa,0,0,0,0,0,0,0,11,38,20,...,12984,60359,401317,422276,639421,488288,707890,557971,3039,32321
5608f200cf,0,0,1,167,536,928,1130,1334,1495,1441,...,1398347,1378177,1322792,1198018,1068464,914794,767811,723110,618212,548407
15b1a0d61e,0,0,1,167,536,928,1130,1334,1495,2,...,274648,167136,185198,252391,215359,251921,224073,209129,182934,229802
4770051790,0,0,1,167,536,928,1130,1334,1495,2,...,274648,553842,4937844,5163063,6530247,4545146,637359,297436,1373250,1958891


In [14]:
merchant_data[merchant_data['count-2033-1']!=0]["count-2033-1"].mean()

45.146153846153844

In [23]:
merchant_data_columns_dict_b50 = merchant_data_columns_dict.copy()
for column in merchant_data_columns_dict.keys():
    average = merchant_data[merchant_data[column]!=0][column].quantile(
    q=0.50,                      # The percentile to calculate
#     axis=1,                     # The axis to calculate the percentile on
#     numeric_only=True,          # To calculate only for numeric columns
    interpolation='linear'      # The type of interpolation to use when the quantile is between 2 values
)
    merchant_data_columns_dict_b50[column] = average
b50_merchant = pd.DataFrame([merchant_data_columns_dict_b50], columns=merchant_data_columns_dict_b50.keys())
b50_merchant['id'] = '50%_bottom'
# This hardly works since there is a long tail problem for payment counts (1 or 2 orders per month for 30% of the bottom merchants)

In [42]:
# Getting merchants that never churn
count_columns_names = [str(x) for x in merchant_data_columns_dict.keys() if "count" in x ]
volume_columns_names = [str(x) for x in merchant_data_columns_dict.keys() if "volume" in x]
count_columns_names
merchant_data_nonchurn = merchant_data[~(merchant_data == 0).any(axis=1)].copy()
merchant_data_nonchurn['min_count'] = merchant_data_nonchurn.loc[:,count_columns_names].min(axis=1)
merchant_data_nonchurn['avg_count'] = merchant_data_nonchurn[count_columns_names].mean(axis=1)
merchant_data_nonchurn['min_volume'] = merchant_data_nonchurn[volume_columns_names].min(axis=1)
merchant_data_nonchurn['avg_volume'] = merchant_data_nonchurn[volume_columns_names].mean(axis=1)
merchant_data_nonchurn['min/avg_count'] = merchant_data_nonchurn['min_count']/merchant_data_nonchurn['avg_count']
merchant_data_nonchurn['min/avg_volume'] = merchant_data_nonchurn['min_volume']/merchant_data_nonchurn['avg_volume']
print(merchant_data_nonchurn['min/avg_count'].mean(axis = 0))
print(merchant_data_nonchurn['min/avg_volume'].mean(axis = 0))
# Insights: min/avg_count = 0.324346, min/avg_volume = 0.272355, use 30% as the indicator for potential default

0.3243463773823513
0.27235566693756574


In [66]:
merchant_data_churnpotential = merchant_data[(merchant_data == 0).any(axis=1)].copy()
merchant_data_churntable = pd.DataFrame(columns = ["id"])
merchant_data_churntable.set_index("id")
merchant_data_churntable_dict = {}
for year in ['2033','2034']:
    for month in range(1,13):
        merchant_data_churntable[year+"-"+str(month)] = 0
        merchant_data_churntable_dict[year+"-"+str(month)] = 0
for index, row in merchant_data_churnpotential.iterrows():
    merchant_data_churntable_dict_copy = merchant_data_churntable_dict.copy()
    for i in range(4,13):
        moving_avg_count = row[ i-3:i].mean()
        moving_avg_volume = row[ i+12-3:i+12].mean()
        merchant_data_churnpotential[str(i)+"-mac"] = moving_avg_count
        merchant_data_churnpotential[str(i)+"-mav"] = moving_avg_volume
        if (int(row[i]) <= moving_avg_count*.3 and int(row[i+12]) <= moving_avg_volume*.3 and moving_avg_count > 0 ):
            merchant_data_churntable_dict_copy[list(merchant_data_churntable_dict_copy)[i-1]] = 1
    merchant_row = pd.DataFrame([merchant_data_churntable_dict_copy], columns=merchant_data_churntable_dict_copy.keys())
    merchant_row['id'] = str(row[0])
    merchant_data_churntable = pd.concat([merchant_data_churntable, merchant_row], axis=0)

In [67]:
merchant_data_churntable.to_csv("churnt.csv")

In [68]:
merchant_data_churnpotential.to_csv("churnpoten.csv")

### References
1. https://chaotic-flow.com/saas-metrics-faqs-what-is-churn/#:~:text=SaaS%20churn%20is%20the%20percentage,important%20parameter%20in%20revenue%20forecasting.
2. https://dataconomy.com/2017/07/churn-predictive-analytics/